# Hotspots "spots adoptés" notebook

The goal of this notebook is to construct a function that return a map of adopted spot by regions, with a filter by environment.

In [3]:
import pandas as pd
pd.set_option("display.max_rows", None)

import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import folium
from shapely.geometry import Point
from folium import plugins
from folium.plugins import MarkerCluster
from branca.colormap import LinearColormap

In [8]:
# Global variables
# csv path
DATA_PATH = 'https://github.com/dataforgoodfr/12_zero_dechet_sauvage/raw/2-nettoyage-et-augmentation-des-donn%C3%A9es/Exploration_visualisation/data/data_zds_enriched.csv'

# Place other global variables here
FRANCE_REGIONS = "regions-avec-outre-mer.geojson"

In [5]:
# Load the enriched dataset
data_zds = pd.read_csv(DATA_PATH)

# Display col names
display(pd.DataFrame({"columns": data_zds.columns}))

,columns
0,ID_RELEVE
1,LIEU_PAYS
2,LIEU_REGION
3,LIEU_DEPT
4,LIEU_VILLE
5,LIEU_CODE_POSTAL
6,LIEU_COORD_GPS
7,LIEU_COORD_GPS_X
8,LIEU_COORD_GPS_Y
9,NOM_ZONE


In [11]:
# Display the column "type de milieu"
data_zds["REGION"].unique()

array(["Provence-Alpes-Côte d'Azur", 'Occitanie', 'Normandie',
       'Auvergne-Rhône-Alpes', 'Île-de-France', 'Nouvelle-Aquitaine',
       'Bretagne', 'Corse', 'Bourgogne-Franche-Comté',
       'Centre-Val de Loire', 'Pays de la Loire', 'Grand Est',
       'Hauts-de-France'], dtype=object)

In [17]:
# Create the function(s) for scalable filtering
def construct_query_string(bound_word = " and ",
                           **params
                           ) -> str:
    """Construct a query string in the right format for the pandas 'query'
    function. The different params are bounded together in the query string with the
    bound word given by default. If one of the params is 'None', it is not
    included in the final query string."""

    # Instanciate query string
    query_string = ""

    # Iterate over the params to construct the query string
    for param_key, param in params.items():
        # Construct the param sub string if the param is not 'None'
        if param:
            query_sub_string = f"{param_key} == '{param}'"

            # Add to the query string
            query_string += f"{query_sub_string}{bound_word}"

    # Strip any remaining " and " at the end of the query string
    return query_string.strip(bound_word)

In [18]:
# Construct the adopted waste spots function
def plot_adopted_waste_spots(data_zds: pd.DataFrame,
                             filter_dict: dict,
                             region_geojson = FRANCE_REGIONS,
                             ) -> folium.Map:
    """Show a folium innteractive map of adopted spots within a selected region,
    filtered by environments of deposit.
    Arguments:
    - data_zds: The waste dataframe
    - filter_dict: dictionary mapping the name of the column in the waste df and the value you want to filter by
    """
    ####################################
    # 1/ Create the waste geodataframe #
    ####################################

    # Create a GeoDataFrame for waste points
    gdf = gpd.GeoDataFrame(
        data_zds,
        geometry = gpd.points_from_xy(data_zds["LIEU_COORD_GPS_X"], data_zds["LIEU_COORD_GPS_Y"]),
        crs = "EPSG:4326"
    )

    # Construct the query string
    query_string = construct_query_string(**filter_dict)

    # Filter the geodataframe by region and by environment
    gdf_filtered = gdf.query(query_string)

    ######################################
    # 2/ Create the regions geodataframe #
    ######################################

    # Unpack the region name
    region = filter_dict["REGION"]

    # Load France regions from a GeoJSON file
    regions = gpd.read_file(region_geojson)
    regions = regions.loc[regions["nom"] == region, :]

    # Filter the region geodataframe for the specified region
    selected_region = regions[regions["nom"].str.lower() == region.lower()]
    if selected_region.empty:
        raise KeyError(f"Region '{region}' not found.")

    ############################
    # 3/ Initialize folium map #
    ############################

    # Initialize a folium map, centered around the mean location of the waste points
    map_center = [gdf_filtered.geometry.y.mean(), gdf_filtered.geometry.x.mean()]
    m = folium.Map(location = map_center, zoom_start = 5)  # Adjust zoom_start as needed for the best initial view

    ######################
    # 4/ Add the markers #
    ######################

    # Use MarkerCluster to manage markers if dealing with a large number of points
    marker_cluster = MarkerCluster().add_to(m)

    # Add each waste point as a marker on the folium map
    for _, row in gdf_filtered.iterrows():
        # Define the marker color: green for adopted spots, red for others
        marker_color = 'darkgreen' if row['SPOT_A1S'] else 'red'
        # Define the icon: check-circle for adopted, info-sign for others
        icon_type = 'check-circle' if row['SPOT_A1S'] else 'info-sign'

        folium.Marker(
            location = [row.geometry.y, row.geometry.x],
            popup = f"Zone: {row['NOM_ZONE']}<br>Date: {row['DATE']}<br>Volume: {row['VOLUME_TOTAL']} litres",
            icon = folium.Icon(color = marker_color, icon = icon_type, prefix = 'fa')
        ).add_to(marker_cluster)

    ##############################
    # 5/ Add the region boundary #
    ##############################

    # Add the region boundary to the map for context
    folium.GeoJson(
        selected_region,
        name = "Region Boundary",
        style_function = lambda feature: {
            'weight': 2,
            'fillOpacity': 0.1,
        }
    ).add_to(m)

    # Return the map
    return m

In [19]:
# Construct the filter dict
# region = "Occitanie"
# environment = "Littoral (terrestre)"

filter_dict = {"REGION": "Occitanie",
               "TYPE_MILIEU": "Littoral (terrestre)"}

# Test the function
plot_adopted_waste_spots(data_zds,
                         filter_dict)